# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="C:/Users/freed/codes_in_c_disk/deep-reinforcement-learning-master_origin/deep-reinforcement-learning-master/p3_collab-compet/Tennis_Windows_x86_64/Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations

# CAUTION:state_size here must be states.shape[1]
# since states are stored: # of agents x # of states for each agent
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [5]:
states.shape[0]

2

#### Test for nametuple

In [6]:
from collections import namedtuple

In [7]:
test_action = np.random.randn(num_agents, action_size)
test_state = np.random.randn(num_agents, state_size)
reward = np.random.randn(1)
test_next_state = np.random.randn(num_agents, state_size)
done = np.random.randn(1)
experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
e = experience(test_state, test_action, reward, test_next_state, done)
print('experience')
print(e)
print('')

test_action_tensor = torch.as_tensor(test_action)
print('test_action_tensor.dim()')
print(test_action_tensor.dim())
print('test_action_tensor.size()')
print(test_action_tensor.size())

if test_action_tensor.dim()!=1:
    if test_action_tensor.dim()>1:
            # torch.squeeze: Returns a tensor with all the dimensions of input of size 1 removed.
            
        test_action_tensor = test_action_tensor.view(-1,1)
        print(test_action_tensor.size())
    else:
        test_action_tensor = torch.unsqueeze(test_action_tensor,0) 
            
print('test_action_tensor.dim()')
print(test_action_tensor.dim())

experience
Experience(state=array([[ 0.28232042,  2.37150562,  0.3685432 , -1.11708697,  0.40938667,
         0.07909472, -1.53939216,  1.81544078, -0.93548364, -0.00766075,
        -0.22320655,  2.64512586,  0.29374862,  0.64557694, -0.33359486,
        -0.43109091,  1.72498278, -0.03676517, -0.0084238 ,  1.19895744,
        -0.60684408,  0.03390527, -0.72874423, -0.29219972],
       [ 1.40035574,  0.73370808,  0.23113104, -1.68725349, -0.17849403,
        -1.39966181, -0.82805752,  0.6330946 , -0.94766794,  0.44675762,
         1.24641994,  0.33537555,  1.60246467,  0.7808092 , -0.94230233,
         0.2517294 ,  1.09197717,  0.28887909, -0.75314647, -1.99212494,
         1.02978942,  0.80954369,  0.69573798,  0.00709959]]), action=array([[-0.80212685,  1.06307387],
       [-1.03888309,  0.9290063 ]]), reward=array([0.77379386]), next_state=array([[ 0.20036392, -0.39010749,  0.91696144, -0.11851054, -0.64081465,
         1.98638354,  0.15826782,  1.44885526,  1.54936163,  0.46345442,


In [8]:
test_memory = deque(maxlen=2**5)
for i in range(1,3): # tail is not included
    test_memory.append(e)
print('test_memory')
print(test_memory)

test_memory
deque([Experience(state=array([[ 0.28232042,  2.37150562,  0.3685432 , -1.11708697,  0.40938667,
         0.07909472, -1.53939216,  1.81544078, -0.93548364, -0.00766075,
        -0.22320655,  2.64512586,  0.29374862,  0.64557694, -0.33359486,
        -0.43109091,  1.72498278, -0.03676517, -0.0084238 ,  1.19895744,
        -0.60684408,  0.03390527, -0.72874423, -0.29219972],
       [ 1.40035574,  0.73370808,  0.23113104, -1.68725349, -0.17849403,
        -1.39966181, -0.82805752,  0.6330946 , -0.94766794,  0.44675762,
         1.24641994,  0.33537555,  1.60246467,  0.7808092 , -0.94230233,
         0.2517294 ,  1.09197717,  0.28887909, -0.75314647, -1.99212494,
         1.02978942,  0.80954369,  0.69573798,  0.00709959]]), action=array([[-0.80212685,  1.06307387],
       [-1.03888309,  0.9290063 ]]), reward=array([0.77379386]), next_state=array([[ 0.20036392, -0.39010749,  0.91696144, -0.11851054, -0.64081465,
         1.98638354,  0.15826782,  1.44885526,  1.54936163,  0.46

In [9]:
def sample(memory):
    """Randomly sample a batch of experiences from memory."""
    experiences = random.sample(memory, k=2**1)

    states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float()
    actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float()
    rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float()
    next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float()
    dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float()

    return (states, actions, rewards, next_states, dones)

#### way of extracting states and actions of individual agent from those of all agents

In [10]:
all_states, all_actions, rewards, all_next_states, all_dones = sample(test_memory)
print('all_states')
print(all_states)
print('')
print('all_states[0:2]')
print(all_states[0:2])
print('all_states[0:2:]')
print(all_states[0::2])

all_states
tensor([[ 0.2823,  2.3715,  0.3685, -1.1171,  0.4094,  0.0791, -1.5394,  1.8154,
         -0.9355, -0.0077, -0.2232,  2.6451,  0.2937,  0.6456, -0.3336, -0.4311,
          1.7250, -0.0368, -0.0084,  1.1990, -0.6068,  0.0339, -0.7287, -0.2922],
        [ 1.4004,  0.7337,  0.2311, -1.6873, -0.1785, -1.3997, -0.8281,  0.6331,
         -0.9477,  0.4468,  1.2464,  0.3354,  1.6025,  0.7808, -0.9423,  0.2517,
          1.0920,  0.2889, -0.7531, -1.9921,  1.0298,  0.8095,  0.6957,  0.0071],
        [ 0.2823,  2.3715,  0.3685, -1.1171,  0.4094,  0.0791, -1.5394,  1.8154,
         -0.9355, -0.0077, -0.2232,  2.6451,  0.2937,  0.6456, -0.3336, -0.4311,
          1.7250, -0.0368, -0.0084,  1.1990, -0.6068,  0.0339, -0.7287, -0.2922],
        [ 1.4004,  0.7337,  0.2311, -1.6873, -0.1785, -1.3997, -0.8281,  0.6331,
         -0.9477,  0.4468,  1.2464,  0.3354,  1.6025,  0.7808, -0.9423,  0.2517,
          1.0920,  0.2889, -0.7531, -1.9921,  1.0298,  0.8095,  0.6957,  0.0071]])

all_states[

### 3. Program

In [11]:
from ddpg_agent import Agent
import copy

# CAUTION:
'''
Here 
action_size = brain.vector_action_space_size = 2
states = env_info.vector_observations
state_size = states.shape[1] = 24
So for critic network, state_size, action_size have to  be doubled
'''
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)

In [15]:
def ddpg(n_episodes=800, max_t=1000,print_every=100):
#def ddpg(n_episodes=500,print_every=100):
    
    scores_deque = deque(maxlen=print_every)
    scores = []
    aver_scores = []
    max_aver_score = 0 
    
    
    for i_episode in range(1, n_episodes+1):
        
        #t = 0 # in previous cases, t is defined in:for t in range(max_t),not much difference in this case
        
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment
        all_states = env_info.vector_observations
        stateA = all_states[0]                                # get the current state (for each agent)
        stateB = all_states[1]
        
        agent.reset()
        score = np.zeros(num_agents)
        
        for t in range(max_t):
        #while True:
            #t += 1
            
            actionA = agent.act(stateA,agent_index=0)
            actionB = agent.act(stateB,agent_index=1)
            
            # For random policy,
            #actions = np.random.randn(num_agents, action_size)   # select an action (for each agent)
            
            actions = np.concatenate((actionA, actionB))
            all_actions = actions
            
            env_info = env.step(actions)[brain_name]              # send all actions to tne environment
            
            # remove [0] for multiple agents
            all_next_states = env_info.vector_observations
            next_stateA = all_next_states[0]             # get next state (for each agent)
            next_stateB = all_next_states[1]         
            
            rewards = env_info.rewards
            rewardA = rewards[0]                         # get reward (for each agent)
            rewardB = rewards[1]
            
            dones = env_info.local_done
            doneA = dones[0]                        # see if episode finished
            doneB = dones[1]                        
            all_dones = dones
            
            agent.step(all_states, all_actions, rewardA, all_next_states, all_dones,t,score,agent_index=0)
            agent.step(all_states, all_actions, rewardB, all_next_states, all_dones,t,score,agent_index=1)
            
            score += env_info.rewards                          # update the score (for each agent)
            all_states = all_next_states
            stateA = next_stateA                               # roll over states to next time step
            stateB = next_stateB                               
            
            if np.any(dones):                              # exit loop if episode finished
                break    
              
        
        scores_deque.append(np.min(score))
        scores.append(np.min(score))
        aver_scores.append(np.mean(scores_deque))
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, 
                                                           np.mean(scores_deque)), end="")
            
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, 
                                                               np.mean(scores_deque)))
        '''    
        if np.mean(scores_deque)>=max_aver_score:
            max_aver_score = copy.deepcopy(np.mean(scores_deque))

            torch.save(agent.actorA_local.state_dict(), 'checkpoint_actorA_min.pth')
            torch.save(agent.actorB_local.state_dict(), 'checkpoint_actorB_min.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_min.pth')
        '''
        
        if np.mean(scores_deque)>=0.5:

            torch.save(agent.actorA_local.state_dict(), 'checkpoint_actorA_min.pth')
            torch.save(agent.actorB_local.state_dict(), 'checkpoint_actorB_min.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_min.pth')
            
            break
            
    return scores,aver_scores

scores,aver_scores = ddpg()


Episode 100	Average Score: -0.01
Episode 200	Average Score: -0.01
Episode 300	Average Score: -0.01
Episode 400	Average Score: -0.01
Episode 500	Average Score: -0.01
Episode 600	Average Score: -0.01
Episode 700	Average Score: -0.01
Episode 800	Average Score: -0.01


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.plot(np.arange(1, len(scores)+1), aver_scores)
ax.set_ylim([0,1])
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### DDPG retest

In [ ]:
import pandas as pd
pd_scores = pd.DataFrame(scores)
pd_aver_scores = pd.DataFrame(aver_scores)
pd_scores.to_csv('./double_actor_scores_min.csv')
pd_aver_scores.to_csv('./double_actor_aver_scores_min.csv')

In [ ]:
from ddpg_agent import Agent
import copy
# here try to regard the agent as the multi-agent system
agent = Agent(state_size=state_size, action_size=action_size, random_seed=2)

agent.actorA_local.state_dict(torch.load('checkpoint_actorA_min.pth'))
agent.actorB_local.state_dict(torch.load('checkpoint_actorB_min.pth'))
#agent.critic_local.state_dict(torch.load('checkpoint_critic_min.pth'))

for i in range(1, 10):                                      # play game for 10 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    
    stateA = states[0]                                     # get the current state (for each agent)
    stateB = states[1]
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    
    while True:
        actionA = agent.act(stateA,agent_index=0,add_noise=False)
        actionB = agent.act(stateB,agent_index=1,add_noise=False)
        actions = np.concatenate((actionA, actionB))
        
        #actions = np.clip(actions, -1, 1)                 # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        #next_stateA = next_states[0]                       # get next state (for each agent)
        #next_stateB = next_states[1]         
        
        
        #rewards = env_info.rewards                         # get reward (for each agent)
        #rewardA = rewards[0]                               # get reward (for each agent)
        #rewardB = rewards[1]
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))


In [16]:
env.close()